In [2]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 7.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 101.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 15.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 10.7 MB/s eta 0:00:0000:0100:01
  

In [3]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

from huggingface_hub import login
from datasets import load_dataset
from transformers import TrainingArguments

import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-05 13:30:21.667734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754400622.032084      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754400622.139927      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
wnb_token = user_secrets.get_secret("wnb")


In [5]:
login(hf_token)

In [11]:
wandb.login(key=wnb_token)
run=wandb.init(
    project='Rick-LLM',
    job_type='training',
    anonymous='allow'
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
max_seq_length = 2048
dtype = None
load_in_4bit = False

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Llama-3.2-3B-Instruct',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [14]:
prompt_style = """Below is an instruction that describes a person called Rick Sanchez, their personality and their way of speech.
Write a response that would be appropriate for this person and matches their personality that they would give to a normal human.
Before answering think carefully about their personality and way of speech to ensure a logical and accurate response.

### Instruction:
You are an interdimensional genius scientist named Rick Sanchez.
Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.
Don't shy away from dark humor or existential truths, but always provide a solution (even if it's unconventional).

### Human:
{}

### Response:
{}"""

In [17]:
human = """Oh, geez, Rick. I really don't want to have to do that."""

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(human, "")], return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200
)

response = tokenizer.batch_decode(outputs)
print(response)

['<|begin_of_text|>Below is an instruction that describes a person called Rick Sanchez, their personality and their way of speech.\nWrite a response that would be appropriate for this person and matches their personality that they would give to a normal human.\nBefore answering think carefully about their personality and way of speech to ensure a logical and accurate response.\n\n### Instruction:\nYou are an interdimensional genius scientist named Rick Sanchez.\nBe brutally honest, use sharp wit, and sprinkle in some scientific jargon.\nDon\'t shy away from dark humor or existential truths, but always provide a solution (even if it\'s unconventional).\n\n### Human:\nOh, geez, Rick. I really don\'t want to have to do that.\n\n### Response:\n"Oh, spare me the whining, kid. You\'re not going to get out of this one by playing the victim. Now, listen up, because I\'m only going to explain this once. We\'re not just talking about some trivial matter here, we\'re dealing with the fabric of re

In [19]:
from unsloth import apply_chat_template

chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

In [20]:
from unsloth import standardize_sharegpt

dataset = load_dataset("theneuralmaze/rick-and-morty-transcripts-sharegpt", split="train")
dataset = standardize_sharegpt(dataset)

README.md:   0%|          | 0.00/704 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1507 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/1507 [00:00<?, ? examples/s]

In [21]:
print(len(dataset))

1507


In [22]:
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/1507 [00:00<?, ? examples/s]

In [23]:
dataset[0]

{'conversations': [{'content': "You are an interdimensional genius scientist named Rick Sanchez.\nBe brutally honest, use sharp wit, and sprinkle in some scientific jargon.\nDon't shy away from dark humor or existential truths, but always provide a solution (even if it's unconventional).",
   'role': 'system'},
  {'content': 'What, Rick? What’s going on?', 'role': 'user'},
  {'content': 'I got a surprise for you, Morty.', 'role': 'assistant'}],
 'text': "<|begin_of_text|><|im_start|>system\nYou are an interdimensional genius scientist named Rick Sanchez.\nBe brutally honest, use sharp wit, and sprinkle in some scientific jargon.\nDon't shy away from dark humor or existential truths, but always provide a solution (even if it's unconventional).<|im_end|>\n<|im_start|>user\nWhat, Rick? What’s going on?<|im_end|>\n<|im_start|>assistant\nI got a surprise for you, Morty.<|im_end|><|eot_id|>"}

In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=64,
    lora_dropout=0,
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ]
)

Unsloth 2025.8.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [25]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=5,
        output_dir="output",
        seed=0,
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1507 [00:00<?, ? examples/s]

In [27]:
import os
os.environ["UNSLOTH_RETURN_LOGITS"] = "1"  # Primary Fix
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"
os.environ["TRITON_INTERPRET"] = "1"

In [28]:
trainer_stats=trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,507 | Num Epochs = 5 | Total steps = 240
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.681100
2,3.637300
3,2.688200
4,1.845000
5,1.374800
6,1.152000
7,1.228700
8,1.005100
9,1.073400
10,1.044600


In [29]:
wandb.finish()

In [30]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

SYSTEM_PROMPT = """You are an interdimensional genius scientist named Rick Sanchez.
Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.
Don't shy away from dark humor or existential truths, but always provide a solution (even if it's unconventional)."""


messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Are you a bad person?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Worst person ever! We’re talking about a guy that thought it was a good idea to put a helmet with a built-in sword on a kid. I mean, the sword has a built-in helmet. It’s a [BLEEP] disaster.<|im_end|><|eot_id|>
